### Anomaly Detection Mini Project
#### Corey Solitaire
`11.09.2020`

<div class="alert alert-block alert-info"><b></b>

#### Imports:

In [17]:

import wrangle
import explore
import numpy as np
import pandas as pd
import math
from sklearn import metrics

from scipy.stats import entropy

import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates #to format dates on our plots
import seaborn as sns

# This is to make sure matplotlib doesn't throw the following error:
# The next line fixes "TypeError: float() argument must be a string or a number, not 'Timestamp' matplotlib"
pd.plotting.register_matplotlib_converters()

<div class="alert alert-block alert-info"><b></b>

### Import wrangle df:

In [2]:
df = wrangle.read_csv()
df = wrangle.wrangle_df(df)
df.head(1)

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2018-01-26 09:55:03,/,1,8,97.105.19.61,2018,1,26,9,Friday,1634276157


In [3]:
df.tail(1)

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2020-11-02 16:48:47,javascript-i/bom-and-dom/bom,763,62,107.192.148.199,2020,11,2,16,Monday,1807783111


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 719459 entries, 2018-01-26 09:55:03 to 2020-11-02 16:48:47
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   page_viewed  719458 non-null  object
 1   user_id      719459 non-null  object
 2   cohort_id    674619 non-null  object
 3   ip           719459 non-null  object
 4   year         719459 non-null  object
 5   month        719459 non-null  object
 6   day          719459 non-null  object
 7   hour         719459 non-null  object
 8   weekday      719459 non-null  object
 9   int_ip       719459 non-null  int64 
dtypes: int64(1), object(9)
memory usage: 60.4+ MB


`Explore Missing user_id Further`

<div class="alert alert-block alert-info"><b></b>

# Questions to Answer:
****
  1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?


  2. Is there a cohort that referred to a lesson significantly more that other cohorts seemed to gloss over? 


  3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students? 


  4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses? Any odd user-agents? 


  5. At some point in the last year, ability for students and alumni to cross-access curriculum (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before? 


  6. What topics are grads continuing to reference after graduation and into their jobs (for each program)? 


  7. Which lessons are least accessed? 


  8. Anything else I should be aware of? 

<div class="alert alert-block alert-info"><b></b>

## First Step is to ID Cohorts:

explore.cohort_id(df, year, month, day, hour)
***

In [ ]:
# 21,Sequoia,2017-09-27,2018-02-15,2
# Cohort ID #21 I think

In [27]:
# 22,Teddy,2018-01-08,2018-05-17,2 
# test = explore.cohort_id(df,2018,1,26,10)
# test.head()

#### Cohort Teddy

In [9]:
teddy = df[df.cohort_id == 22]
len(teddy.user_id.unique())

35

In [24]:
# 23,Ulysses,2018-03-05,2018-07-19,2
# test = explore.cohort_id(df,2018,3,5,10)
# test.head()

#### Cohort Ulysses

In [23]:
ulysses = df[df.cohort_id == 17]
len(ulysses.user_id.unique())

13

In [30]:
# 24,Voyageurs,2018-05-29,2018-10-11,2
# test = explore.cohort_id(df,2018,5,29,10)
# test.head()

#### Cohort Voyagers

In [29]:
voyagers = df[df.cohort_id == 23]
len(voyagers.user_id.unique())

33

In [33]:
# 25,Wrangell,2018-07-23,2018-11-29,2
# test = explore.cohort_id(df,2018,7,23,10)
# test.head()

#### Cohort Wranglell

In [32]:
wranglell = df[df.cohort_id == 24]
len(wranglell.user_id.unique())

34

In [36]:
# 26,Xanadu,2018-09-17,2019-02-08,2
# test = explore.cohort_id(df,2018,9,17,10)
# test.head()

#### Cohort Xanadu

In [35]:
xanadu = df[df.cohort_id == 25]
len(xanadu.user_id.unique())

26

In [40]:
# 27,Yosemite,2018-11-05,2019-04-03,2
# test = explore.cohort_id(df,2018,11,5,10)
# test.head(20)

#### Cohort Yosemite

In [39]:
yosemite = df[df.cohort_id == 26]
len(yosemite.user_id.unique())

20

In [6]:
# 28,Staff,2014-02-04,2014-02-04,2

# 29,Zion,2019-01-22,2019-06-04,2

# 30,Ada,2019-02-04,2019-06-16,3

# 31,Andromeda,2019-03-18,2019-07-30,2

# 32,Betelgeuse,2019-05-28,2019-10-08,2

# 33,Ceres,2019-07-15,2019-12-11,2

# 34,Bayes,2019-08-19,2020-01-30,3

# 51,Deimos,2019-09-16,2020-02-27,2

# 52,Europa,2019-11-04,2020-04-17,2

# 53,Fortuna,2020-01-13,2020-06-23,2

# 55,Curie,2020-02-03,2020-07-07,3

# 56,Apex,2020-02-24,2020-07-29,2

# 57,Ganymede,2020-03-23,2020-08-20,2

# 58,Hyperion,2020-05-26,2020-11-10,2

# 59,Darden,2020-07-13,2021-01-12,3

# 60,Io,2020-07-20,2021-01-21,2

# 61,Bash,2020-07-20,2021-01-21,2

# 62,Jupiter,2020-09-21,2021-03-30,2

# 132,Kalypso,2020-11-02,2021-05-04,2